In [26]:
import zenml
from zenml.steps import step
import gradio as gr
from pathlib import Path
import wandb

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchvision.models import resnet18

print(f'zenml=={zenml.__version__}')
print(f'gradio=={gr.__version__}')
print(f'wandb=={wandb.__version__}')

zenml==0.20.5
gradio==3.6
wandb==0.13.4


In [22]:
PROJECT_PATH = Path.cwd().parent
WEIGHT_DIR = PROJECT_PATH / 'weights'
CHECKPOINT_PATH = WEIGHT_DIR / 'mri.ckpt'
STAGED_MODEL_FILENAME = 'staged_mri.pt'
LOG_DIR = PROJECT_PATH / 'logs'
STAGED_MODEL_TYPE = 'deployment-demo'
STAGE_MODEL_NAME = 'staged_mri_demo'

The MRI model code (copied from `../../mri/src/train.py`)

In [7]:
class MRIModel(pl.LightningModule):

    def __init__(self, lr=0.001):
        super().__init__()
        self.lr = lr
        self.resnet = resnet18()
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7),  # change input channel to be 1 instead of 3 
                                      stride=(2, 2), padding=(3, 3), bias=False)
        # add a linear layer at the end for transfer learning
        self.linear = nn.Linear(in_features=self.resnet.fc.out_features,
                                out_features=5)
        self.save_hyperparameters()  # log hyperparameters

    # optionally, define a forward method
    def forward(self, xs):
        y_hats = self.resnet(xs)
        y_hats = self.linear(y_hats)
        return y_hats  # we like to just call the model's forward method
    
    def training_step(self, batch, batch_idx):
        xs, ys = batch
        y_hats = self.forward(xs)
        loss = F.binary_cross_entropy_with_logits(y_hats, ys)
        self.log("train_loss", loss, prog_bar=True, on_epoch=True, on_step=True)
        return loss

    def validation_step(self, batch, batch_idx):
        xs, ys = batch
        y_hats = self.forward(xs)
        loss = F.binary_cross_entropy_with_logits(y_hats, ys)
        self.log("val_loss", loss, prog_bar=True, on_epoch=True, on_step=True)
    
    # def test_step(self, xs, batch_idx):
    #     y_hats = self.resnet(xs)
    #     y_hats = self.linear(y_hats)
    #     return y_hats

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

Load the trained model from checkpoint ((copied from `../../mri/weights/epoch=*.ckpt`))

In [11]:
model = MRIModel.load_from_checkpoint(CHECKPOINT_PATH)
model.eval()

MRIModel(
  (resnet): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

Save the model to torchscript in the staging directory

In [18]:
def save_model_to_torchscript(model, directory):
    scripted_model = model.to_torchscript(method="script", file_path=None)
    path = Path(directory) / STAGED_MODEL_FILENAME
    torch.jit.save(scripted_model, path)

save_model_to_torchscript(model, WEIGHT_DIR)

Upload the model to `weights & biases`

In [23]:
def upload_staged_model(staged_at, from_directory):
    staged_at.add_file(Path(from_directory) / STAGED_MODEL_FILENAME)
    wandb.log_artifact(staged_at)

In [25]:
with wandb.init(
    job_type='stage', entity="multi-modal-fsdl2022", project='deployment', dir=LOG_DIR, 
):
    staged_at = wandb.Artifact(STAGE_MODEL_NAME, type=STAGED_MODEL_TYPE)
    upload_staged_model(staged_at, from_directory=WEIGHT_DIR)


wandb: Currently logged in as: khoaguin (multi-modal-fsdl2022). Use `wandb login --relogin` to force relogin


Running our more portable model via a CLI (following `lab7`)